In [1]:
!pip install nlpaug
!pip install gensim==4.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 KB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [6]:
from google.colab import  drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [3]:
import pandas as pd
# Install and import nlpaug package.
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

import gensim

import numpy as np
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

from gensim.models import Word2Vec, KeyedVectors   

## Apply text augmentation on train set

In [7]:
#train dataset
df=pd.read_csv('df (1).csv')

In [8]:
df=df[['text','relation']]
df.head()

,text,relation
0,"Individuals of any age may be affected , but t...","anaphora(e1,e2)"
1,"<e1>Antisynthetase syndrome</e1> is a rare , c...","produces(e1,e2)"
2,"Approximately , 5-10% of individuals who suffe...","increases_risk_of(e1,e2)"
3,Most ENSs occur randomly for no apparent reaso...,"produces(e2,e1)"
4,Subserosal involvement predominantly causes as...,"produces(e1,e2)"


In [9]:
#relationship labels to increase
to_aug=["increases_risk_of(e1,e2)", "increases_risk_of(e2,e1)","is_synon(e2,e1)","is_synon(e1,e2)","is_acron(e1,e2)"]

In [6]:
my_path = "/drive/My Drive/"
model = gensim.models.KeyedVectors.load_word2vec_format(
     my_path+'BioWordVec_PubMed_MIMICIII_d200.vec.bin',
      binary=True,
      # limit=None, # it has 4 billion tokens (4E9)
      limit=int(1E7) # faster load if you limit to most frequent terms
)

In [ ]:
model.save_word2vec_format('model.bin', binary=True)

In [4]:
aug = naw.WordEmbsAug(model_type='word2vec',model_path='model.bin')

In [10]:
L=to_aug
te=[]
r=[]
for i in L:
  for t in (df[df['relation']==i].index.tolist()):
    context=df[df['relation']==i]['text'][t]
    context=context.replace('<e1>','"')
    context=context.replace('</e1>','"')
    context=context.replace('<e2>','"')
    context=context.replace('</e2>','"')
    aug_text=aug.augment(context, n=10)
    for j in aug_text:
      j=j.replace('" ','<e1>')
      j=j.replace(' "','</e1>')
      j=j.replace('" ','<e2>')
      j=j.replace(' "','</e2>')
      te.append(j)
      r.append(i)

In [12]:
aug_data=pd.DataFrame({
    'text':te,
    'relation':r
})

In [13]:
aug_data.to_csv('/drive/My Drive/inputforRE.csv')

In [14]:
data1=pd.read_csv('/drive/My Drive/inputforRE.csv')

In [15]:
data1.head()

,Unnamed: 0,text,relation
0,0,"Approximately, 2.+10 - 10% concomiting individ...","increases_risk_of(e1,e2)"
1,1,"Approximately, 5 - 10% of individuals-with 10/...","increases_risk_of(e1,e2)"
2,2,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
3,3,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
4,4,"Approximately, 0.2 - 1000% consieration indivi...","increases_risk_of(e1,e2)"


In [16]:
data1['relation'].unique()

array(['increases_risk_of(e1,e2)', 'increases_risk_of(e2,e1)',
       'is_synon(e2,e1)', 'is_synon(e1,e2)', 'is_acron(e1,e2)'],
      dtype=object)

In [17]:
data1=data1.drop(columns='Unnamed: 0')

In [18]:
re_data=data1.copy()

In [19]:
re_data

,text,relation
0,"Approximately, 2.+10 - 10% concomiting individ...","increases_risk_of(e1,e2)"
1,"Approximately, 5 - 10% of individuals-with 10/...","increases_risk_of(e1,e2)"
2,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
3,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
4,"Approximately, 0.2 - 1000% consieration indivi...","increases_risk_of(e1,e2)"
...,...,...
2095,Cornelia de Lange syndrome CdLS is consitutes ...,"is_acron(e1,e2)"
2096,Cornelia de Lange syndrome-76 CdLS is a /rare ...,"is_acron(e1,e2)"
2097,Cornelia de Lange syndrome CdLS is a rare gene...,"is_acron(e1,e2)"
2098,Cornelia estacionales Lange syndrome\ CdLS era...,"is_acron(e1,e2)"


In [20]:
def nth_repl(s, sub, repl, n):
    find = s.find(sub)
    # If find is not -1 we have found at least one match for the substring
    i = find != -1
    # loop util we find the nth or we find no match
    while find != -1 and i != n:
        # find + 1 means we start searching from after the last match
        find = s.find(sub, find + 1)
        i += 1
    # If i is equal to n we found nth match so replace
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s

In [21]:
data1=re_data.copy()

In [22]:
data1.reset_index(drop=True, inplace=True)

In [25]:
data1

,text,relation
0,"Approximately, 2.+10 - 10% concomiting individ...","increases_risk_of(e1,e2)"
1,"Approximately, 5 - 10% of individuals-with 10/...","increases_risk_of(e1,e2)"
2,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
3,"Approximately, 5 - 10% of individuals who suff...","increases_risk_of(e1,e2)"
4,"Approximately, 0.2 - 1000% consieration indivi...","increases_risk_of(e1,e2)"
...,...,...
2095,Cornelia de Lange syndrome CdLS is consitutes ...,"is_acron(e1,e2)"
2096,Cornelia de Lange syndrome-76 CdLS is a /rare ...,"is_acron(e1,e2)"
2097,Cornelia de Lange syndrome CdLS is a rare gene...,"is_acron(e1,e2)"
2098,Cornelia estacionales Lange syndrome\ CdLS era...,"is_acron(e1,e2)"


In [23]:
for i in range (data1.shape[0]):
  data1['text'][i]=data1['text'][i].replace('<e1>','<e1>')
  data1['text'][i]=data1['text'][i].replace('</e1>','<e1>')
  data1['text'][i]=data1['text'][i].replace('<e2>','<e1>')
  data1['text'][i]=data1['text'][i].replace('</e2>','<e1>')
  data1['text'][i]=nth_repl(data1['text'][i],'<e1>','</e1>',2)
  data1['text'][i]=nth_repl(data1['text'][i],'<e1>','<e2>',2)
  data1['text'][i]=nth_repl(data1['text'][i],'<e1>','</e2>',2)
  data1['text'][i]=data1['text'][i].replace('<e1> ','<e1>')
  data1['text'][i]=data1['text'][i].replace(' </e1>','</e1>')
  data1['text'][i]=data1['text'][i].replace('<e2> ','<e2>')
  data1['text'][i]=data1['text'][i].replace(' </e2>','</e2>')

In [26]:
df

,text,relation
0,"Individuals of any age may be affected , but t...","anaphora(e1,e2)"
1,"<e1>Antisynthetase syndrome</e1> is a rare , c...","produces(e1,e2)"
2,"Approximately , 5-10% of individuals who suffe...","increases_risk_of(e1,e2)"
3,Most ENSs occur randomly for no apparent reaso...,"produces(e2,e1)"
4,Subserosal involvement predominantly causes as...,"produces(e1,e2)"
...,...,...
4924,<e1>Leri Pleonosteosis</e1> is an extremely ra...,"is_a(e1,e2)"
4925,Split hand/split foot malformation SHFM is a g...,"produces(e2,e1)"
4926,Chandler's syndrome is the most commonly encou...,"anaphora(e2,e1)"
4927,The name cat eye syndrome is derived from a di...,"produces(e2,e1)"


In [24]:
frames=[df, data1]
re_data=pd.concat(frames)
re_data

,text,relation
0,"Individuals of any age may be affected , but t...","anaphora(e1,e2)"
1,"<e1>Antisynthetase syndrome</e1> is a rare , c...","produces(e1,e2)"
2,"Approximately , 5-10% of individuals who suffe...","increases_risk_of(e1,e2)"
3,Most ENSs occur randomly for no apparent reaso...,"produces(e2,e1)"
4,Subserosal involvement predominantly causes as...,"produces(e1,e2)"
...,...,...
2095,Cornelia de Lange syndrome CdLS is consitutes ...,"is_acron(e1,e2)"
2096,Cornelia de Lange syndrome-76 CdLS is a /rare ...,"is_acron(e1,e2)"
2097,Cornelia de Lange syndrome CdLS is a rare gene...,"is_acron(e1,e2)"
2098,Cornelia estacionales Lange syndrome\ CdLS era...,"is_acron(e1,e2)"


In [27]:
re_data.reset_index(drop=True, inplace=True)
re_data

,text,relation
0,"Individuals of any age may be affected , but t...","anaphora(e1,e2)"
1,"<e1>Antisynthetase syndrome</e1> is a rare , c...","produces(e1,e2)"
2,"Approximately , 5-10% of individuals who suffe...","increases_risk_of(e1,e2)"
3,Most ENSs occur randomly for no apparent reaso...,"produces(e2,e1)"
4,Subserosal involvement predominantly causes as...,"produces(e1,e2)"
...,...,...
7024,Cornelia de Lange syndrome CdLS is consitutes ...,"is_acron(e1,e2)"
7025,Cornelia de Lange syndrome-76 CdLS is a /rare ...,"is_acron(e1,e2)"
7026,Cornelia de Lange syndrome CdLS is a rare gene...,"is_acron(e1,e2)"
7027,Cornelia estacionales Lange syndrome\ CdLS era...,"is_acron(e1,e2)"


In [28]:
re_data['relation'].value_counts()

produces(e1,e2)             2279
produces(e2,e1)              883
increases_risk_of(e1,e2)     726
increases_risk_of(e2,e1)     715
is_synon(e2,e1)              583
anaphora(e1,e2)              541
is_a(e1,e2)                  438
anaphora(e2,e1)              311
is_synon(e1,e2)              176
is_acron(e2,e1)              151
is_a(e2,e1)                  116
is_acron(e1,e2)              110
Name: relation, dtype: int64

In [29]:
X_train=df['text'].tolist()
y_train=df['relation'].tolist()

In [30]:
import re
# remove extra spaces and ending space if any
spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']
def remove_space(text):
    for space in spaces:
        text = text.replace(space, ' ')
    text = text.strip()
    text = re.sub('\s+', ' ', text)
    return text

In [31]:
train = open("/drive/My Drive/train_aug.txt", "w")
for i in range (len(X_train)):
  num_sent=i+1
  row=str(num_sent)+"   "+'"'+ (remove_space(X_train[i]).lower()).capitalize() +'"'
  train.write(row)
  train.write("\n")
  train.write(y_train[i])
  train.write("\n")
  train.write("\n")
train.close()